# "US Presidential Candidate Tweets"
> "A visualization of recent tweets from candidates in the 2020 US Presidential Election"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [Data Visualization]
- image: 
- hide: false
- search_exclude: true

Test content.